# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096881


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:35,  3.40s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<01:02,  2.39s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:43,  1.73s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:30,  1.25s/it]

...

Rendering models:  73%|███████▎  | 22/30 [00:08<00:01,  5.83it/s]

Rendering models:  80%|████████  | 24/30 [00:08<00:00,  6.57it/s]

Rendering models:  90%|█████████ | 27/30 [00:09<00:00,  8.14it/s]

Rendering models: 100%|██████████| 30/30 [00:09<00:00,  9.59it/s]

not-logged-in-0687f3923f81ac7bd40a    0.010678
Jnursssmith5263                       0.001392
Alexander00                           0.000808
jnarayanbvg                           0.019739
cameronbrooke                         0.242504
not-logged-in-c6917d9b19f41e6c2995    0.029093
jaksonA                               0.005089
salmi.a01                             0.018617
lsautter                              0.000941
not-logged-in-9a45911eb71184fe7c7f    0.003682
Arlo-M                                0.002611
not-logged-in-73b973978dc7c9e2e5ac    0.000726
not-logged-in-f82a6d15a1dafa9bbfae    0.000404
jmartinez468                          0.003717
databanana                            0.000382
dumdum                                0.002063
peakscience7                          0.031585
pangeli5                              0.000331
Tyler_Arirama                         0.001113
Lavadude                              0.027460
not-logged-in-f0338f45b1f60a64aa34    0.002954
Tessaje04    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())